In [1]:
#import the neccessary modules
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Activation
import numpy as np
import pandas as pd 
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import zipfile

In [2]:
#creating variable
docs = ['Awesome',
        'Fantastic',
        'Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = np.array([1,1,1,1,1,1,1,0,0,0,0,0])


In [3]:
#tokinizer
t = Tokenizer()
t.fit_on_texts(docs)

In [4]:
# Vocabulary details
print("Word Index:", t.word_index)  
print("Word Counts:", t.word_counts)  
print("Total Documents:", t.document_count)

Word Index: {'work': 1, 'done': 2, 'good': 3, 'effort': 4, 'poor': 5, 'awesome': 6, 'fantastic': 7, 'well': 8, 'great': 9, 'nice': 10, 'excellent': 11, 'weak': 12, 'not': 13, 'could': 14, 'have': 15, 'better': 16}
Word Counts: OrderedDict([('awesome', 1), ('fantastic', 1), ('well', 1), ('done', 2), ('good', 2), ('work', 3), ('great', 1), ('effort', 2), ('nice', 1), ('excellent', 1), ('weak', 1), ('poor', 2), ('not', 1), ('could', 1), ('have', 1), ('better', 1)])
Total Documents: 12


In [5]:
vocal_size = len(t.word_index) +1
vocal_size

17

In [6]:
encoding_text = t.texts_to_sequences(docs)
encoding_text

[[6],
 [7],
 [8, 2],
 [3, 1],
 [9, 4],
 [10, 1],
 [11],
 [12],
 [5, 4],
 [13, 3],
 [5, 1],
 [14, 15, 2, 16]]

In [7]:
max_leng = 4
pedding_encoing_text = pad_sequences(encoding_text, maxlen=max_leng, padding="post")
pedding_encoing_text

array([[ 6,  0,  0,  0],
       [ 7,  0,  0,  0],
       [ 8,  2,  0,  0],
       [ 3,  1,  0,  0],
       [ 9,  4,  0,  0],
       [10,  1,  0,  0],
       [11,  0,  0,  0],
       [12,  0,  0,  0],
       [ 5,  4,  0,  0],
       [13,  3,  0,  0],
       [ 5,  1,  0,  0],
       [14, 15,  2, 16]])

In [8]:
filepath = r"C:\Users\HP\Desktop\NLP\glove.6B.zip"
with zipfile.ZipFile(filepath, 'r') as f:
    #print(f.namelist())
    embedded_dict = {}
    with f.open('glove.6B.100d.txt') as file:
        for line in file:
            token = line.split()
            word = token[0]
            vector = np.asarray(token[1:], dtype='float32')
            embedded_dict[word] =vector
            
         

In [9]:
print('Loaded %s word vectors.' % len(embedded_dict))

Loaded 400000 word vectors.


In [10]:
embedded_matrix = np.zeros((vocal_size, 100))
for word,i in t.word_index.items():
    embedded_index = embedded_dict.get(word)
    if embedded_index is not None:
        embedded_matrix[i] = embedded_index
    

In [11]:
# define model
model = Sequential()
model.add(Embedding(vocal_size, 100, weights=[embedded_matrix], input_length=4, trainable=False))
#flatten to one dimension
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
# summarize the model
print(model.summary())
# fit the model
model.fit(pedding_encoing_text, labels, epochs=50, verbose=2)
# evaluate the model
loss, accuracy = model.evaluate(pedding_encoing_text, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            1700      
                                                                 
 flatten (Flatten)           (None, 400)               0         
                                                                 
 dense (Dense)               (None, 1)                 401       
                                                                 
Total params: 2,101
Trainable params: 401
Non-trainable params: 1,700
_________________________________________________________________
None
Epoch 1/50
1/1 - 1s - loss: 0.6931 - accuracy: 0.4167 - 1s/epoch - 1s/step
Epoch 2/50
1/1 - 0s - loss: 0.6931 - accuracy: 0.5833 - 6ms/epoch - 6ms/step
Epoch 3/50
1/1 - 0s - loss: 0.6930 - accuracy: 0.5833 - 8ms/epoch - 8ms/step
Epoch 4/50
1/1 - 0s - loss: 0.6929 - accuracy: 0.5833 - 7ms/epoch - 7ms/step
Epoch